In [1]:
import pytket as tk
from pytket import passes as tkp
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter as print_circ
import numpy as np
from pytket import OpType

In [36]:
def h_gadgetisation2(circ: Circuit):
    n_qubits = circ.n_qubits #original # of qubits
    h = circ.n_gates_of_type(OpType.H) # of H gates

    new_circ = Circuit(n_qubits+h) #Define a replacement circuit


    for cmd in circ.get_commands():
        qubit_list = cmd.qubits # accesing the qubit of cmd
        if cmd.op.type == OpType.H:
            q1 = qubit_list[0].index[0] # we want the number
            qa = q1 + (n_qubits-1) # ancilla qubit
            new_circ.H(qa)
            new_circ.S(q1).S(qa).CX(q1, qa).Sdg(qa).CX(qa,q1).CX(q1,qa) # If cmd is a H gate, do H-gadgetisation
            #new_circ.add_barrier()
            

        else:
            # Otherwise, apply the gate as usual.
            new_circ.add_gate(cmd.op.type, cmd.op.params, qubit_list)
        

    return new_circ

In [38]:
DecompseZPass = tkp.CustomPass(h_gadgetisation2) # Define our pass

test_circ = tk.Circuit(5).H(1).CX(2,3).CX(2,1).CY(1,2).H(2).Z(3)
print_circ(test_circ)

DecompseZPass.apply(test_circ) # Apply our pass to the test Circuit

test_circ.get_commands()
#print_circ(test_circ)

[S q[1];,
 CX q[2], q[3];,
 H q[5];,
 H q[6];,
 Z q[3];,
 S q[5];,
 S q[6];,
 CX q[1], q[5];,
 Sdg q[5];,
 CX q[5], q[1];,
 CX q[1], q[5];,
 CX q[2], q[1];,
 CY q[1], q[2];,
 S q[2];,
 CX q[2], q[6];,
 Sdg q[6];,
 CX q[6], q[2];,
 CX q[2], q[6];]

In [31]:
test1 = tk.Circuit(5).H(1).CX(2,3).CX(2,1).CY(1,2).H(2).Z(3)
print_circ(test1)
test2 = tk.Circuit(5).H(1).CX(2,3)
test1.append(test2)

print_circ(test1)


In [39]:
series = test1.to_dict()

In [40]:
series

{'bits': [],
 'commands': [{'args': [['q', [1]]], 'op': {'type': 'H'}},
  {'args': [['q', [2]], ['q', [3]]], 'op': {'type': 'CX'}},
  {'args': [['q', [2]], ['q', [1]]], 'op': {'type': 'CX'}},
  {'args': [['q', [3]]], 'op': {'type': 'Z'}},
  {'args': [['q', [1]], ['q', [2]]], 'op': {'type': 'CY'}},
  {'args': [['q', [1]]], 'op': {'type': 'H'}},
  {'args': [['q', [2]]], 'op': {'type': 'H'}},
  {'args': [['q', [2]], ['q', [3]]], 'op': {'type': 'CX'}}],
 'created_qubits': [],
 'discarded_qubits': [],
 'implicit_permutation': [[['q', [0]], ['q', [0]]],
  [['q', [1]], ['q', [1]]],
  [['q', [2]], ['q', [2]]],
  [['q', [3]], ['q', [3]]],
  [['q', [4]], ['q', [4]]]],
 'phase': '0.0',
 'qubits': [['q', [0]], ['q', [1]], ['q', [2]], ['q', [3]], ['q', [4]]]}

In [43]:
recuper = Circuit()
recuperation = recuper.from_dict(series)

print_circ(recuperation)